# Segmenting and Clustering Neighborhoods in Toronto


## Web scrapping of wikipedia " List of postal codes of Canada" page

### import of the pakages
### I  use beutifullsoup to do the webscrapping (the first html table)

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

headers = requests.utils.default_headers()

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

#print(soup.prettify())

tabs =soup.find_all("table")# i only take the first table in the page
tab=tabs[0]

### Read the rows and fill a dictionnary (the key is the postal code)

In [6]:
dic={}#i create a dictionnary from the html table, the key is the postal code

#loops on the tr and td html tags
for row in tab.find_all('tr'):
   
    columns = row.find_all('td')
   
    postalCode=""
    borough=""
    neighborhood=""

    tds=[0,0,0]
    ind=0

    for column in columns:
       tds[ind]= column.get_text().rstrip()#\n suppress
       ind+=1
    
    #i skip the header row
    if ind==0:
        continue  
  
    #i read the values 
    postalCode=tds[0]
    borough=tds[1]

    if(tds[2]!='Not assigned'):
        neighborhood=tds[2]
    else:
        neighborhood=borough

    #filling the dictionnary
    if postalCode in dic :
        #the postalcode already exist in the dictionnary
        dic[postalCode]=dic[postalCode][1]+','+neighborhood
    else:
        if(borough!='Not assigned'):
            dic[postalCode]=[borough,neighborhood]
        


In [9]:
#building the dataframe
df=pd.DataFrame.from_dict(dic,columns=['Borough','Neighborhood'],  orient='index')
df['PostalCode']=df.index
df.reset_index(drop=True,inplace=True)
df = df.reindex(columns = ['PostalCode', 'Borough','Neighborhood'])
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 3)